<a href="https://colab.research.google.com/github/R06880/Covid19-Detection-Using-CNN/blob/main/Covid19_Detection_Using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount(‘/content/gdrive’)

SyntaxError: ignored

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
/content/gdrive/My Drive/CovidDataset/Train

In [4]:
# import libraries
# we will preprocess the images using the preprocessing modules from keras
# we will import all layers and all models from Keras
# we import PIL to load images

from PIL import Image
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import * 
print('Imported Successfully!')

Imported Successfully!


In [5]:
# 

train_datagen = image.ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = image.ImageDataGenerator(rescale = 1./255)

print('Created the Data Generator Objects.')

Created the Data Generator Objects.


In [7]:
train_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/CovidDataset/Train',target_size=(224,224),batch_size=32, class_mode="binary")
val_generator = test_datagen.flow_from_directory('/content/gdrive/My Drive/CovidDataset/Val',target_size=(224,224),batch_size=32, class_mode="binary")


Found 224 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation="relu",input_shape=(224,224,3)))
model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam",metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 128)       0

In [9]:
hist = model.fit(train_generator, epochs = 6, validation_data=val_generator, validation_steps=2)

Epoch 1/6
7/7 [==============================] - 148s 20s/step - loss: 4.1166 - accuracy: 0.4884 - val_loss: 0.6837 - val_accuracy: 0.8833
Epoch 2/6
7/7 [==============================] - 76s 11s/step - loss: 0.5738 - accuracy: 0.7288 - val_loss: 0.3300 - val_accuracy: 0.9333
Epoch 3/6
7/7 [==============================] - 76s 11s/step - loss: 0.3271 - accuracy: 0.8449 - val_loss: 0.1990 - val_accuracy: 0.9667
Epoch 4/6
7/7 [==============================] - 76s 11s/step - loss: 0.2403 - accuracy: 0.9089 - val_loss: 0.1894 - val_accuracy: 0.9667
Epoch 5/6
7/7 [==============================] - 76s 11s/step - loss: 0.2300 - accuracy: 0.9241 - val_loss: 0.5050 - val_accuracy: 0.9667
Epoch 6/6
7/7 [==============================] - 76s 11s/step - loss: 0.2350 - accuracy: 0.9353 - val_loss: 0.0753 - val_accuracy: 0.9667
